For work with **File** and **Directory** you can choose between either of two classes: 
- ***Static classes*** : `File` and `Directory`  
- ***Instance-method classes*** : `FileInfo` and `DirectoryInfo`

there’s a **static** class called `Path`. This `does nothing` to **files** or **directories**; instead, it provides string **manipulation methods** for `filenames` and `directory paths`. `Path` also assists with `temporary files`.

### The File Class

***File*** is a static class whose `methods all accept a filename`. The `filename` can be either `relative to the current directory` or `fully qualified` with a directory.

In [ ]:
using System.IO;
using System.Security.AccessControl;

public interface test{
bool Exists (string path); // Returns true if the file is present
void Delete (string path);
void Copy (string sourceFileName, string destFileName);
void Move (string sourceFileName, string destFileName);
void Replace (string sourceFileName, string destinationFileName,
                                    string destinationBackupFileName);
FileAttributes GetAttributes (string path);
void SetAttributes (string path, FileAttributes fileAttributes);
void Decrypt (string path);
void Encrypt (string path);
DateTime GetCreationTime (string path); // UTC versions are
DateTime GetLastAccessTime (string path); // also provided.
DateTime GetLastWriteTime (string path);
void SetCreationTime (string path, DateTime creationTime);
void SetLastAccessTime (string path, DateTime lastAccessTime);
void SetLastWriteTime (string path, DateTime lastWriteTime);
FileSecurity GetAccessControl (string path);
FileSecurity GetAccessControl (string path,
AccessControlSections includeSections);
void SetAccessControl (string path, FileSecurity fileSecurity);
}

Here are all the members of the `FileAttribute` **enum** that `GetAttributes` returns:  
  
`Archive`, `Compressed`, `Device`, `Directory`, `Encrypted`,  
`Hidden`, `IntegritySystem`, `Normal`, `NoScrubData`, `NotContentIndexed`,  
`Offline`, `ReadOnly`, `ReparsePoint`, `SparseFile`, `System`, `Temporary`

In [ ]:
using System.IO;

//Members in FileAttributes enum are combinable. Here’s how to toggle a single file attribute
//without upsetting the rest:

string filePath = "test.txt";
FileAttributes fa = File.GetAttributes (filePath);
if ((fa & FileAttributes.ReadOnly) != 0)
{
    // Use the exclusive-or operator (^) to toggle the ReadOnly flag
    fa ^= FileAttributes.ReadOnly;
    File.SetAttributes (filePath, fa);
}
// Now we can delete the file, for instance:
File.Delete (filePath);

#### File security

This feature is `Windows-only`  
The `FileSecurity` class allow you to **query** and **change** the `OS permissions` assigned to users and roles

In [ ]:
//we list a file’s existing permissions and then assign Write permission
//to the “Users” group

using System;
using System.IO;
using System.Security.AccessControl;
using System.Security.Principal;
void ShowSecurity (FileSecurity sec)
{
    AuthorizationRuleCollection rules = sec.GetAccessRules (true, true, typeof (NTAccount));
    foreach (FileSystemAccessRule r in rules.Cast<FileSystemAccessRule>()
            .OrderBy (rule => rule.IdentityReference.Value))
    {
        // e.g., MyDomain/Joe
        Console.WriteLine ($" {r.IdentityReference.Value}");
        // Allow or Deny: e.g., FullControl
        Console.WriteLine ($" {r.FileSystemRights}: {r.AccessControlType}");
    }
}

var file = "sectest.txt";
File.WriteAllText (file, "File security test.");

var sid = new SecurityIdentifier (WellKnownSidType.BuiltinUsersSid, null);
string usersAccount = sid.Translate (typeof (NTAccount)).ToString();

Console.WriteLine ($"User: {usersAccount}");

FileSecurity sec = new FileSecurity (file,
                            AccessControlSections.Owner |
                            AccessControlSections.Group |
                            AccessControlSections.Access);
Console.WriteLine ("AFTER CREATE:");
ShowSecurity(sec); // BUILTIN\Users doesn't have Write permission

sec.ModifyAccessRule (AccessControlModification.Add,
    new FileSystemAccessRule (usersAccount, FileSystemRights.Write,
        AccessControlType.Allow),
    out bool modified);

Console.WriteLine ("AFTER MODIFY:");
ShowSecurity (sec); // BUILTIN\Users has Write permission


### The Directory Class

In [ ]:
using System.IO;

public interface test{
string GetCurrentDirectory ();
void SetCurrentDirectory (string path);

DirectoryInfo CreateDirectory (string path);
DirectoryInfo GetParent (string path);
string GetDirectoryRoot (string path);

string[] GetLogicalDrives(); // Gets mount points on Unix

// The following methods all return full paths:
string[] GetFiles (string path);
string[] GetDirectories (string path);
string[] GetFileSystemEntries (string path);

IEnumerable<string> EnumerateFiles (string path);
IEnumerable<string> EnumerateDirectories (string path);
IEnumerable<string> EnumerateFileSystemEntries (string path);
}

### FileInfo and DirectoryInfo

The static methods on `File` and `Directory` are convenient for executing a **single** `file` or `directory` **operation**. If you need to `call a series of methods` in a row, the `FileInfo` and `DirectoryInfo` classes provide an object model that makes the job easier.

In [ ]:
using System.IO;

static string TestDirectory => @"C:\Temp";

Directory.CreateDirectory (TestDirectory);

FileInfo fi = new FileInfo (Path.Combine (TestDirectory, "FileInfo.txt"));

Console.WriteLine (fi.Exists); // false

using (TextWriter w = fi.CreateText())
    w.Write ("Some text");

Console.WriteLine (fi.Exists); // false (still)
fi.Refresh();
Console.WriteLine (fi.Exists); // true

Console.WriteLine (fi.Name); // FileInfo.txt
Console.WriteLine (fi.FullName); // c:\temp\FileInfo.txt (Windows)
// /tmp/FileInfo.txt (Unix)

Console.WriteLine (fi.DirectoryName); // c:\temp (Windows)
// /tmp (Unix)

Console.WriteLine (fi.Directory.Name); // temp
Console.WriteLine (fi.Extension); // .txt
Console.WriteLine (fi.Length); // 9


fi.Encrypt();
fi.Attributes ^= FileAttributes.Hidden; // (Toggle hidden flag)
fi.IsReadOnly = true;

Console.WriteLine (fi.Attributes); // ReadOnly,Archive,Hidden,Encrypted
Console.WriteLine (fi.CreationTime); // 3/09/2019 1:24:05 PM

fi.MoveTo (Path.Combine (TestDirectory, "FileInfoX.txt"));

DirectoryInfo di = fi.Directory;
Console.WriteLine (di.Name); // temp or tmp
Console.WriteLine (di.FullName); // c:\temp or /tmp
Console.WriteLine (di.Parent.FullName); // c:\ or /
di.CreateSubdirectory ("SubFolder");

In [ ]:
using System.IO;
//Here’s how to use DirectoryInfo to enumerate files and subdirectories:

DirectoryInfo di = new DirectoryInfo (@"e:\photos");

foreach (FileInfo fi in di.GetFiles ("*.jpg"))
    Console.WriteLine (fi.Name);

foreach (DirectoryInfo subDir in di.GetDirectories())
    Console.WriteLine (subDir.FullName);

### Path

The ***static*** `Path` class defines methods and fields for working with `paths` and `filenames`.

In [2]:
using System.IO;
//Assuming this setup code:

string dir = @"c:\mydir"; // or /mydir
string file = "myfile.txt";
string path = @"c:\mydir\myfile.txt"; // or /mydir/myfile.txt

Directory.SetCurrentDirectory (@"f:\Learning"); // or /demo

Console.WriteLine(Directory.GetCurrentDirectory()); //k:\demo\ or /demo
Console.WriteLine(Path.IsPathRooted (file)); // False
Console.WriteLine(Path.IsPathRooted (path)); // True
Console.WriteLine(Path.GetPathRoot (path)); // c:\ or /
Console.WriteLine(Path.GetDirectoryName (path));  // c:\mydir or /mydir

Console.WriteLine(Path.GetFileName (path)); // myfile.txt
Console.WriteLine(Path.GetFullPath (file)); //k:\demo\myfile.txt
                         //or /demo/myfile.txt
Console.WriteLine(Path.Combine (dir, file)); //c:\mydir\myfile.txt
                         //or /mydir/myfile.txt

/*File Extentions:*/
Console.WriteLine(Path.HasExtension (file)); //True
Console.WriteLine(Path.GetExtension (file)); //.txt
Console.WriteLine(Path.GetFileNameWithoutExtension (file)); //myfile
Console.WriteLine(Path.ChangeExtension (file, ".log")); //myfile.log

/*Separators and characters:*/
Console.WriteLine(Path.DirectorySeparatorChar); // \ or /
Console.WriteLine(Path.AltDirectorySeparatorChar); // /
Console.WriteLine(Path.PathSeparator); // ; or:
Console.WriteLine(Path.VolumeSeparatorChar); //: or /
Console.WriteLine(Path.GetInvalidPathChars()); // chars 0 to 31 and "<>|eor 0
Console.WriteLine(Path.GetInvalidFileNameChars()); // chars 0 to 31 and "<>|:*?\/ or 0 and /

/*Temporary files*/
Console.WriteLine(Path.GetTempPath()); // <local user folder>\Temp or /tmp/
Console.WriteLine(Path.GetRandomFileName()); // d2dwuzjf.dnp
Console.WriteLine(Path.GetTempFileName()); // <local user folder>\Temp\tmp14B.tmp
                                            //or /tmp/tmpubSUYO.tmp

f:\Learning
False
True
c:\
c:\mydir
myfile.txt
f:\Learning\myfile.txt
c:\mydir\myfile.txt
True
.txt
myfile
myfile.log
\
/
;
:
| 	

"<>| 	
:*?\/


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>متد <code>Combine</code> در کلاس <code>Path</code> بسیار مفید است زیرا به شما این امکان را می‌دهد که یک دایرکتوری و یک نام فایل - یا دو دایرکتوری - را بدون نیاز به بررسی دستی وجود یا عدم وجود یک جداکننده مسیر (path separator) ترکیب کنید. این متد به طور خودکار از جداکننده مسیر صحیح برای سیستم‌عامل استفاده می‌کند. همچنین دارای اورلودهایی است که تا چهار دایرکتوری و/یا نام فایل را می‌پذیرند.</p>

</div>

In [ ]:
string dir1 = @"C:\Users";
string dir2 = "Public";
string dir3 = "Documents";
string filename = "example.txt";
string fullPath = Path.Combine(dir1, dir2, dir3, filename);
Console.WriteLine(fullPath);  // نتیجه: C:\Users\Public\Documents\example.txt

`GetFullPath` **converts** a ***path relative*** to the current directory to an ***absolute path***. It accepts values such as `..\..\file.txt`.

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">

<h5> متد <code>Path.GetRandomFileName</code></h5>
<p>این متد یک نام فایل یکتا به فرمت 8.3 کاراکتری (هشت کاراکتر برای نام فایل و سه کاراکتر برای پسوند) برمی‌گرداند، بدون اینکه فایل واقعی را ایجاد کند. این نام‌ها به طور تصادفی تولید می‌شوند و برای مواردی که نیاز به یک نام فایل یکتا دارید، اما نیازی به ایجاد فایل ندارید، مفید هستند.</p>

<h5>2. متد <code>Path.GetTempFileName</code></h5>
<p>این متد یک نام فایل موقت تولید می‌کند که از یک شمارنده خودافزاینده استفاده می‌کند. این شمارنده بعد از تولید 65,000 فایل تکرار می‌شود. پس از تولید نام فایل، این متد یک فایل با حجم صفر بایت در دایرکتوری موقت سیستم ایجاد می‌کند. این متد برای مواردی که نیاز به یک فایل موقت واقعی دارید و می‌خواهید مطمئن شوید که فایل ایجاد شده است، مفید است.</p>

</div>

### Special Folders

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>در دات‌نت، برای دسترسی به پوشه‌های خاص سیستم عامل مانند My Documents، Program Files، Application Data و غیره، می‌توانید از متد <code>GetFolderPath</code> در کلاس <code>Environment</code> استفاده کنید. این متد مسیرهای این پوشه‌ها را برمی‌گرداند. برای این کار، باید از <code>Environment.SpecialFolder</code> که یک <code>enum</code> است و مقادیر آن شامل تمامی دایرکتوری‌های خاص در ویندوز می‌باشد، استفاده کنید.</p>
</div>

In [3]:
string myDocPath = Environment.GetFolderPath(Environment.SpecialFolder.MyDocuments);
Console.WriteLine($"My Documents Path: {myDocPath}");

My Documents Path: C:\Users\bahar\Documents


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<h4>SpecialFolder Enum</h4>
<p><code>Environment.SpecialFolder</code> شامل مقادیر متعددی برای دایرکتوری‌های خاص ویندوز است. برخی از این مقادیر عبارتند از:</p>
<ul><li><code>AdminTools</code></li><li><code>ApplicationData</code></li><li><code>CommonProgramFiles</code></li><li><code>Desktop</code></li><li><code>Favorites</code></li><li><code>Fonts</code></li><li><code>History</code></li><li><code>InternetCache</code></li><li><code>LocalApplicationData</code></li><li><code>MyDocuments</code></li><li><code>ProgramFiles</code></li><li><code>Recent</code></li><li><code>SendTo</code></li><li><code>StartMenu</code></li><li><code>System</code></li><li><code>Templates</code></li></ul>

<h4>پوشه‌های خاص برای ذخیره داده‌های برنامه</h4>
<ol>
<li><p><strong>ApplicationData</strong>:</p>
<ul>
<li><strong>مسیر:</strong> <code>Environment.SpecialFolder.ApplicationData</code></li><li><strong>کاربرد:</strong> این پوشه برای ذخیره تنظیماتی که با کاربر در شبکه جابجا می‌شوند، استفاده می‌شود. اگر پروفایل‌های جابجایی (roaming profiles) در دامنه شبکه فعال باشند، این داده‌ها با کاربر در شبکه جابجا می‌شوند.</li></ul>
</li>

<li>
<p><strong>LocalApplicationData</strong>:</p><ul><li><strong>مسیر:</strong> <code>Environment.SpecialFolder.LocalApplicationData</code></li><li><strong>کاربرد:</strong> این پوشه برای ذخیره داده‌هایی که جابجا نمی‌شوند و مخصوص کاربر وارد شده هستند، استفاده می‌شود. این داده‌ها فقط بر روی سیستم فعلی کاربر ذخیره می‌شوند.</li></ul>
</li>
<li>
<p><strong>CommonApplicationData</strong>:</p><ul><li><strong>مسیر:</strong> <code>Environment.SpecialFolder.CommonApplicationData</code></li><li><strong>کاربرد:</strong> این پوشه برای ذخیره داده‌هایی که بین تمامی کاربران سیستم به اشتراک گذاشته می‌شوند، استفاده می‌شود. این داده‌ها برای تمامی کاربران سیستم در دسترس هستند.</li></ul>
</li>
</ol>
</div>

In [ ]:
using System.IO;

string appName = "MyCoolApplication";

string localAppDataPath = Path.Combine(
    Environment.GetFolderPath(Environment.SpecialFolder.ApplicationData),
    appName);

if (!Directory.Exists(localAppDataPath))
{
    Directory.CreateDirectory(localAppDataPath);
}

Console.WriteLine($"Application Data Path: {localAppDataPath}");

<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p><strong>محدودیت‌های CommonApplicationData</strong>:</p>

<ul><li>اگر برنامه شما توسط کاربری با دسترسی‌های مدیریتی (administrative elevation) اجرا شود و پوشه‌ها و فایل‌هایی در <code>CommonApplicationData</code> ایجاد کند، ممکن است کاربران دیگر که دسترسی‌های محدود دارند، نتوانند این فایل‌ها را تغییر دهند یا جایگزین کنند. برای حل این مشکل، می‌توانید در زمان نصب برنامه، پوشه مورد نظر را با دسترسی‌های مناسب برای همه کاربران ایجاد کنید.</li></ul>

<p><strong>استفاده از پوشه base directory برنامه</strong>:</p>

<ul><li>شما می‌توانید از دایرکتوری اصلی برنامه برای ذخیره فایل‌های تنظیمات و لاگ‌ها استفاده کنید، که با استفاده از <code>AppDomain.CurrentDomain.BaseDirectory</code> قابل دسترسی است. اما این روش توصیه نمی‌شود، زیرا سیستم عامل ممکن است بعد از نصب اولیه برنامه، به برنامه شما اجازه نوشتن در این پوشه را ندهد، مگر اینکه با دسترسی‌های مدیریتی اجرا شود.</li></ul>
</div>

### Querying Volume Information

In [4]:
//You can query the drives on a computer with the DriveInfo class

DriveInfo c = new DriveInfo ("C"); // Query the C: drive.
                                    // On Unix: /
long totalSize = c.TotalSize; // Size in bytes.
long freeBytes = c.TotalFreeSpace; // Ignores disk quotas.
long freeToMe = c.AvailableFreeSpace; // Takes quotas into account.

foreach (DriveInfo d in DriveInfo.GetDrives()) // All defined drives.
                                                // On Unix: mount points
{
    Console.WriteLine (d.Name); // C:\
    Console.WriteLine (d.DriveType); // Fixed
    Console.WriteLine (d.RootDirectory); // C:\
    if (d.IsReady) // If the drive is not ready, the following two
                    // properties will throw exceptions:
    {
        Console.WriteLine (d.VolumeLabel); // The Sea Drive
        Console.WriteLine (d.DriveFormat); // NTFS
    }
}

C:\
Fixed
C:\

NTFS
D:\
Fixed
D:\
Google Drive
FAT32
F:\
Fixed
F:\

NTFS
G:\
Fixed
G:\
New Volume
NTFS


`DriveType` is an ***enum*** with the following values:  
`Unknown`, `NoRootDirectory`, `Removable`, `Fixed`, `Network`, `CDRom`, `Ram`

### Catching Filesystem Events

The `FileSystemWatcher` class lets you ***monitor a directory*** (and optionally, subdirectories) for `activity`.

***FileSystemWatcher*** has `events` that `fire` when **files** or **subdirectories**
are `created`, `modified`, `renamed`, and `deleted`, as well as when `their attributes change`.

In [ ]:
using System.IO;
using System.Runtime.InteropServices;

void Watch (string path, string filter, bool includeSubDirs)
{
    using (var watcher = new FileSystemWatcher (path, filter))
    {
        watcher.Created += FileCreatedChangedDeleted;
        watcher.Changed += FileCreatedChangedDeleted;
        watcher.Deleted += FileCreatedChangedDeleted;
        watcher.Renamed += FileRenamed;
        watcher.Error += FileError;
        watcher.IncludeSubdirectories = includeSubDirs;
        watcher.EnableRaisingEvents = true;
        Console.WriteLine ("Listening for events - press <enter> to end");
        Console.ReadLine();
    }
    // Disposing the FileSystemWatcher stops further events from firing.
}

void FileCreatedChangedDeleted (object o, FileSystemEventArgs e)
    => Console.WriteLine ("File {0} has been {1}", e.FullPath, e.ChangeType);

void FileRenamed (object o, RenamedEventArgs e)
    => Console.WriteLine ("Renamed: {0}->{1}", e.OldFullPath, e.FullPath);

void FileError (object o, ErrorEventArgs e)
    => Console.WriteLine ("Error: " + e.GetException().Message);

string GetTestDirectory() =>
    RuntimeInformation.IsOSPlatform (OSPlatform.Windows)
        ? @"C:\Temp"
        : "/tmp";


`IncludeSubdirectories` applies ***recursively***. So, if you create a FileSystemWatcher on `C:\` with `IncludeSubdirectories true`, its events will fire when a file or directory changes `anywhere` on the `hard drive`.